# Knowledge Distillation in PyTorch



## Basic Setup


In [1]:
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import models
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10

print(f"PyTorch Version: {torch.__version__}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device used: {device.type}")
debug = False
print(f"Debug mode: {debug}")

PyTorch Version: 2.8.0.dev20250319+cu128
Device used: cuda
Debug mode: False


## Load Dataset

In [2]:
# # Use 224×224 resize for ResNet compatibility
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor()
# ])
# transform = models.ResNet50_Weights.IMAGENET1K_V2.transforms()

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],  # CIFAR-10 means
                         std=[0.2023, 0.1994, 0.2010])
])

# Load full CIFAR-10 train set
full_trainset = CIFAR10(root='./data', train=True, download=True, transform=transform)

# Calculate split sizes for train and validation sets
train_size = int(0.9 * len(full_trainset))
val_size = len(full_trainset) - train_size

# Perform split
train_subset, val_subset = random_split(full_trainset, [train_size, val_size])
print(f"Train samples: {train_size}")
print(f"Validation samples: {val_size}")

# Create DataLoaders
train_loader = DataLoader(train_subset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=128, shuffle=False)

# CIFAR-10 test set and loader for accuracy evaluation
test_set = CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_set, batch_size=128, shuffle=False)
print(f"Test samples: {len(test_set)}")


Train samples: 45000
Validation samples: 5000
Test samples: 10000


## Define Models

In [3]:
# Setup teacher and student wrapper
def setup_models(device):
    # Teacher: ResNet50 pretrained on ImageNet, re-headed for CIFAR-10
    teacher = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
    teacher.fc = nn.Linear(2048, 10)
    teacher = teacher.to(device)

    # Student: ResNet18 from scratch
    student = models.resnet18(weights=None)
    student.fc = nn.Linear(512, 10).to(device)
    student = student.to(device)

    student_channels = [64, 128, 256, 512]
    teacher_channels = [256, 512, 1024, 2048]
    proj_layers = [
        FeatureProjector(in_c, out_c).to(device)
        for in_c, out_c in zip(student_channels, teacher_channels)
    ]

    student_wrapper = StudentWrapper(student, proj_layers).to(device)
    return teacher, student_wrapper

## Wrapping the Student Model with Projections

When using intermediate supervision (feature matching), the student’s feature maps often don't match the teacher’s in shape. To fix this, we wrap the student model and its feature projection layers into a single StudentWrapper module:

In [4]:
# Feature projector to match student -> teacher feature shapes
class FeatureProjector(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.proj = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x, target_shape):
        if x.shape[2:] != target_shape[2:]:
            x = F.adaptive_avg_pool2d(x, output_size=target_shape[2:])
        return self.proj(x)

# Wrapper for the student model with projection layers
class StudentWrapper(nn.Module):
    def __init__(self, student_model, proj_layers):
        super().__init__()
        self.model = student_model
        self.projections = nn.ModuleList(proj_layers)

    def forward(self, x):
        features = []
        x = self.model.conv1(x)
        x = self.model.bn1(x)
        x = self.model.relu(x)
        x = self.model.maxpool(x)
        for i, block in enumerate([self.model.layer1, self.model.layer2, self.model.layer3, self.model.layer4]):
            x = block(x)
            features.append(x)
        pooled = F.adaptive_avg_pool2d(x, (1, 1))
        flat = torch.flatten(pooled, 1)
        logits = self.model.fc(flat)
        return logits, features

    def project_features(self, features, target_shapes):
        return [
            proj(s_feat, t_shape)
            for s_feat, t_shape, proj in zip(features, target_shapes, self.projections)
        ]

# Extract teacher logits and intermediate features
def extract_features(model, x, layers=[1, 2, 3, 4]):
    features = []
    x = model.conv1(x)
    x = model.bn1(x)
    x = model.relu(x)
    x = model.maxpool(x)
    for i, block in enumerate([model.layer1, model.layer2, model.layer3, model.layer4]):
        x = block(x)
        if (i + 1) in layers:
            features.append(x)

    # Final classification head
    pooled = F.adaptive_avg_pool2d(x, (1, 1))  # [B, C, 1, 1]
    flat = torch.flatten(pooled, 1)            # [B, C]
    logits = model.fc(flat)                    # [B, 10]
    return logits, features


## Setup models and optimizer

In [5]:
teacher, student_wrapper = setup_models(device)
optimizer = torch.optim.Adam(student_wrapper.parameters(), lr=1e-3)


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100.0%


## Define Train Function

In [6]:
def train_teacher(teacher, loader, epochs, tag, lr=1e-3, save_path="model.pth", silent=False):
    """
    Trains a model with Adam and cross-entropy loss.
    Loads from save_path if it exists.
    """

    if os.path.exists(save_path) and debug:
        if not silent:
            print(f"Model already trained. Loading from {save_path}")
        teacher.load_state_dict(torch.load(save_path))
        return teacher

    # no saved model found. training from given model state

    optimizer = torch.optim.Adam(teacher.parameters(), lr=1e-3)
    teacher.train()

    for epoch in range(epochs):
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            logits, _ = extract_features(teacher, inputs)

            loss = F.cross_entropy(logits, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if not silent:
            print(f"({tag})\tEpoch {epoch+1}: loss={loss.item():.4f}")
            # print(f"[Teacher] Epoch {epoch+1}: Loss = {loss.item():.4f}")

            evaluate_accuracy(teacher, val_loader, f"Epoch {epoch+1}")
            teacher.train()

    if save_path:
        torch.save(teacher.state_dict(), save_path)
        if not silent:
            print(f"Training complete. Model saved to {save_path}")
            print("Saved fine-tuned teacher.")

    return teacher

# Function to evaluate accuracy given loader
def evaluate_accuracy(model, dataloader, tag):
    """
    Evaluate and print accuracy given loader
    """
    model.eval()
    model.to(device)
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    accuracy = correct / total
    print(f"Accuracy ({tag}): {accuracy*100:.2f}%")
    return accuracy

## Fine-Tune the Teacher

In [7]:
# Train the teacher on CIFAR-10 for a few epochs
epochs = 20
teacher = train_teacher(teacher, train_loader, epochs=epochs, tag="Fine-tuning teacher", save_path="tuned_pretrained_resnet50_on_CIFAR10.pth")


(Fine-tuning teacher)	Epoch 1: loss=0.4818
Accuracy (Epoch 1): 81.60%
(Fine-tuning teacher)	Epoch 2: loss=0.6269
Accuracy (Epoch 2): 81.70%
(Fine-tuning teacher)	Epoch 3: loss=0.2588
Accuracy (Epoch 3): 82.72%
(Fine-tuning teacher)	Epoch 4: loss=0.2500
Accuracy (Epoch 4): 82.88%
(Fine-tuning teacher)	Epoch 5: loss=0.1956
Accuracy (Epoch 5): 83.52%
(Fine-tuning teacher)	Epoch 6: loss=0.2880
Accuracy (Epoch 6): 84.22%
(Fine-tuning teacher)	Epoch 7: loss=0.1356
Accuracy (Epoch 7): 84.82%
(Fine-tuning teacher)	Epoch 8: loss=0.2171
Accuracy (Epoch 8): 84.50%
(Fine-tuning teacher)	Epoch 9: loss=0.0864
Accuracy (Epoch 9): 83.72%
(Fine-tuning teacher)	Epoch 10: loss=0.1164
Accuracy (Epoch 10): 84.64%
(Fine-tuning teacher)	Epoch 11: loss=0.1246
Accuracy (Epoch 11): 83.88%
(Fine-tuning teacher)	Epoch 12: loss=0.0730
Accuracy (Epoch 12): 84.68%
(Fine-tuning teacher)	Epoch 13: loss=0.0834
Accuracy (Epoch 13): 85.04%
(Fine-tuning teacher)	Epoch 14: loss=0.2676
Accuracy (Epoch 14): 83.54%
(Fine-tuni

## Intermediate Distillation with Projections

## Distillation Loss

In [8]:
# Combine soft and hard targets using KL divergence and cross-entropy
# T = temperature, alpha = weighting between soft and hard losses
def distillation_loss(student_logits, teacher_logits, targets, T=5.0, alpha=0.7):

    # Soft target loss (teacher softmax vs student softmax)
    soft_targets = F.kl_div(
        F.log_softmax(student_logits / T, dim=1),
        F.softmax(teacher_logits / T, dim=1),
        reduction='batchmean'
    ) * (T * T)

    # Hard label loss
    hard_loss = F.cross_entropy(student_logits, targets)
    return alpha * soft_targets + (1 - alpha) * hard_loss


## Evaluation Functions

In [9]:
# Function to count trainable parameters
def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Function to measure average inference latency over multiple runs
def measure_latency(model, input_size=(1, 3, 32, 32), device='cuda', repetitions=50):
    model.eval()
    inputs = torch.randn(input_size).to(device)
    with torch.no_grad():
        # Warm-up
        for _ in range(10):
            _ = model(inputs)
        # Measure
        times = []
        for _ in range(repetitions):
            start = time.time()
            _ = model(inputs)
            end = time.time()
            times.append(end - start)
    return (sum(times) / repetitions) * 1000  # ms


## Train the Student via Distillation

In [10]:
def training_step(inputs, labels, teacher, student_wrapper, optimizer, device):
    inputs, labels = inputs.to(device), labels.to(device)

    with torch.no_grad():
        teacher_logits, teacher_feats = extract_features(teacher, inputs)

    student_logits, student_feats = student_wrapper(inputs)
    projected_feats = student_wrapper.project_features(student_feats, [t.shape for t in teacher_feats])
    feat_loss = sum(F.mse_loss(p, t.detach()) for p, t in zip(projected_feats, teacher_feats))

    loss = distillation_loss(student_logits, teacher_logits, labels) + 0.1 * feat_loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()

In [11]:
# Train the student using the teacher's output as soft targets
teacher.eval()  # Teacher in eval mode

if debug:
  num_epochs = 1
else:
  num_epochs = 25

best_val_acc = 0.0
save_path = "student_distilled.pth"

# Reduce LR if validation loss doesn't improve for 3 epochs
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

for epoch in range(num_epochs):
    student_wrapper.train()
    running_loss = 0
    for inputs, labels in train_loader:
        loss = training_step(inputs, labels, teacher, student_wrapper, optimizer, device)
        running_loss += loss

    val_acc = evaluate_accuracy(student_wrapper.model, val_loader, "student")
    print(f"[(Training student)\tEpoch {epoch+1}] Loss = {running_loss/len(train_loader):.4f} | Val Acc = {val_acc*100:.2f}%")
    scheduler.step(loss)

    # Save best
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(student_wrapper.state_dict(), save_path)
        print("✅ New best model saved.")

# load best checkpoint
student_wrapper.load_state_dict(torch.load(save_path))
student = student_wrapper.model

Accuracy (student): 60.02%
[(Training student)	Epoch 1] Loss = 8.7847 | Val Acc = 60.02%
✅ New best model saved.
Accuracy (student): 65.92%
[(Training student)	Epoch 2] Loss = 5.9411 | Val Acc = 65.92%
✅ New best model saved.
Accuracy (student): 69.38%
[(Training student)	Epoch 3] Loss = 4.8069 | Val Acc = 69.38%
✅ New best model saved.
Accuracy (student): 71.84%
[(Training student)	Epoch 4] Loss = 4.0791 | Val Acc = 71.84%
✅ New best model saved.
Accuracy (student): 74.46%
[(Training student)	Epoch 5] Loss = 3.4702 | Val Acc = 74.46%
✅ New best model saved.
Accuracy (student): 71.80%
[(Training student)	Epoch 6] Loss = 2.9380 | Val Acc = 71.80%
Accuracy (student): 74.02%
[(Training student)	Epoch 7] Loss = 2.5594 | Val Acc = 74.02%
Accuracy (student): 74.18%
[(Training student)	Epoch 8] Loss = 2.2837 | Val Acc = 74.18%
Accuracy (student): 74.72%
[(Training student)	Epoch 9] Loss = 1.9847 | Val Acc = 74.72%
✅ New best model saved.
Accuracy (student): 74.84%
[(Training student)	Epoch 10

## Model Comparison Code

In [12]:
# Compare size, latency, and accuracy
teacher_params = count_params(teacher)
student_params = count_params(student)

teacher_latency = measure_latency(teacher, device=device)
student_latency = measure_latency(student, device=device)

teacher_acc = evaluate_accuracy(teacher, test_loader, "teacher on testset")
student_acc = evaluate_accuracy(student, test_loader, "student on testset")

print(f"Teacher Params: {teacher_params / 1e6:.2f}M")
print(f"Student Params: {student_params / 1e6:.2f}M")
print(f"Teacher Latency: {teacher_latency:.2f} ms")
print(f"Student Latency: {student_latency:.2f} ms")
print(f"Teacher Test Accuracy: {teacher_acc * 100:.2f}%")
print(f"Student Test Accuracy: {student_acc * 100:.2f}%")


Accuracy (teacher on testset): 85.10%
Accuracy (student on testset): 79.24%
Teacher Params: 23.53M
Student Params: 11.18M
Teacher Latency: 3.82 ms
Student Latency: 1.54 ms
Teacher Test Accuracy: 85.10%
Student Test Accuracy: 79.24%


In [13]:
# Bad-Student: ResNet18 training from scratch on its own, re-headed for CIFAR-10
bad_student = models.resnet18(weights=None)
bad_student.fc = nn.Linear(512, 10).to(device)
bad_student = bad_student.to(device)


In [14]:
epochs = 20
bad_student = train_teacher(bad_student, train_loader, epochs=epochs, tag="Bad-student", save_path="bad_student.pth")


(Bad-student)	Epoch 1: loss=1.0124
Accuracy (Epoch 1): 58.06%
(Bad-student)	Epoch 2: loss=0.8091
Accuracy (Epoch 2): 63.10%
(Bad-student)	Epoch 3: loss=0.8210
Accuracy (Epoch 3): 68.06%
(Bad-student)	Epoch 4: loss=0.7438
Accuracy (Epoch 4): 68.74%
(Bad-student)	Epoch 5: loss=0.8731
Accuracy (Epoch 5): 71.18%
(Bad-student)	Epoch 6: loss=0.7033
Accuracy (Epoch 6): 72.70%
(Bad-student)	Epoch 7: loss=0.3875
Accuracy (Epoch 7): 71.32%
(Bad-student)	Epoch 8: loss=0.3319
Accuracy (Epoch 8): 72.06%
(Bad-student)	Epoch 9: loss=0.3094
Accuracy (Epoch 9): 73.80%
(Bad-student)	Epoch 10: loss=0.0464
Accuracy (Epoch 10): 73.28%
(Bad-student)	Epoch 11: loss=0.3143
Accuracy (Epoch 11): 72.76%
(Bad-student)	Epoch 12: loss=0.1305
Accuracy (Epoch 12): 72.06%
(Bad-student)	Epoch 13: loss=0.2509
Accuracy (Epoch 13): 73.52%
(Bad-student)	Epoch 14: loss=0.1958
Accuracy (Epoch 14): 73.06%
(Bad-student)	Epoch 15: loss=0.0951
Accuracy (Epoch 15): 72.90%
(Bad-student)	Epoch 16: loss=0.0546
Accuracy (Epoch 16): 7

In [15]:
bad_student_acc = evaluate_accuracy(bad_student, test_loader, "bad_student on testset")


Accuracy (bad_student on testset): 73.79%


In [16]:
epochs = 5
bad_student = train_teacher(bad_student, train_loader, epochs=epochs, tag="Bad-student", save_path="bad_student.pth")


(Bad-student)	Epoch 1: loss=0.0844
Accuracy (Epoch 1): 73.80%
(Bad-student)	Epoch 2: loss=0.0631
Accuracy (Epoch 2): 72.42%
(Bad-student)	Epoch 3: loss=0.0556
Accuracy (Epoch 3): 73.86%
(Bad-student)	Epoch 4: loss=0.1145
Accuracy (Epoch 4): 73.42%
(Bad-student)	Epoch 5: loss=0.0202
Accuracy (Epoch 5): 74.10%
Training complete. Model saved to bad_student.pth
Saved fine-tuned teacher.
